#### First notebook used for data collection, was used before migrating into jobs

In [1]:
%pip install yfinance

  Using cached yfinance-0.2.40-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.2.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.4 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached frozendict-2.4.4-py312-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.5-py3-none-any.whl
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached charset_normalize

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import boto3
import os

In [3]:
%mkdir -p data/raw

In [4]:
sp500_ticker = "^GSPC"
filename = "data/raw/sp500.csv"

end = datetime.now()
start = end - pd.Timedelta(days=30)

full_data = pd.DataFrame()
while start < end:
    next_time = start + pd.Timedelta(days=7)
    if next_time > end:
        next_time = end

    data = yf.download(sp500_ticker, start=start, end=next_time, interval="1m")

    full_data = pd.concat([full_data, data])
    start = next_time

full_data.to_csv("../" + filename)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
lag = 30
inference_filename = "data/raw/sp500-inference.csv"

end = datetime.now()
start = end - pd.Timedelta(days=3)

data = yf.download(sp500_ticker, start=start, end=next_time, interval="1m")
df = pd.DataFrame(data)

df = df.tail(lag + 1)

df.to_csv("../" + inference_filename)

[*********************100%%**********************]  1 of 1 completed


In [17]:
BUCKET_NAME = "team1-index-predictor-bucket"

In [18]:
s3 = boto3.client("s3")

s3.upload_file("../" + filename, BUCKET_NAME, filename)
s3.upload_file("../" + inference_filename, BUCKET_NAME, inference_filename)

In [20]:
s3.list_objects(Bucket=BUCKET_NAME)["Contents"]

[{'Key': 'data/raw/sp500.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 7, 19, 40, tzinfo=tzlocal()),
  'ETag': '"da61e834eae58ae078dd7ec42eeb5ace"',
  'Size': 916048,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}}]